In [3]:
# 动态规划解决方案

# 定义生产各阶段的成本
detection_cost_part1 = 2  # 零配件1检测成本
detection_cost_part2 = 3  # 零配件2检测成本
detection_cost_product = 3  # 成品检测成本
assembly_cost = 6  # 装配成本
revenue_per_product = 56  # 合格产品收益
replacement_loss = 6  # 退货损失
reputation_loss = 5  # 企业信誉损失
scrap_cost = 5  # 拆解成本

# 假设次品率和检测概率
part1_defect_rate = 0.1  # 零配件1次品率
part2_defect_rate = 0.1  # 零配件2次品率
product_defect_rate = 0.1  # 成品次品率

In [4]:

# 动态规划递推公式，考虑检测与不检测的两种情况
def dp_solve():
    # 状态空间：D1, D2, D3, D4 (是否检测零配件1, 零配件2, 成品，是否拆解)
    dp = {}

    # 定义递归的动态规划函数
    def dp_decision(D1, D2, D3, D4):
        if (D1, D2, D3, D4) in dp:
            return dp[(D1, D2, D3, D4)]

        # 检测零配件的成本
        part1_defects = part1_defect_rate if D1 == 1 else 0  # 零配件1次品率
        part2_defects = part2_defect_rate if D2 == 1 else 0  # 零配件2次品率

        # 检测成品的成本
        product_defects = product_defect_rate if D3 == 1 else 0

        # 合格产品收益与不合格产品的处理
        revenue = (1 - product_defects) * revenue_per_product
        loss = product_defects * (replacement_loss + reputation_loss)

        # 装配后的成品检测成本
        cost = (D1 * detection_cost_part1 + D2 * detection_cost_part2 + 
                D3 * detection_cost_product + assembly_cost - revenue + loss)

        # 拆解决定
        if D4 == 1:
            # 如果选择拆解，增加拆解成本
            cost += scrap_cost
        
        dp[(D1, D2, D3, D4)] = cost
        return cost

    # 遍历所有可能的决策组合，找到成本最小的策略
    min_cost = float('inf')
    best_decision = None
    for D1 in [0, 1]:
        for D2 in [0, 1]:
            for D3 in [0, 1]:
                for D4 in [0, 1]:
                    cost = dp_decision(D1, D2, D3, D4)
                    if cost < min_cost:
                        min_cost = cost
                        best_decision = (D1, D2, D3, D4)
    
    return best_decision, min_cost

# 调用动态规划函数求解最优决策
best_decision, min_cost = dp_solve()
print(f"最优决策组合: {best_decision}, 最小成本: {min_cost}")


最优决策组合: (0, 0, 0, 0), 最小成本: -50


In [5]:
import numpy as np

# 蒙特卡洛模拟参数
num_simulations = 10000

# 定义次品率的概率分布
part1_defect_rate_dist = np.random.uniform(0.05, 0.2, num_simulations)  # 零配件1次品率分布
part2_defect_rate_dist = np.random.uniform(0.05, 0.2, num_simulations)  # 零配件2次品率分布
product_defect_rate_dist = np.random.uniform(0.05, 0.2, num_simulations)  # 成品次品率分布

def monte_carlo_simulation():
    total_costs = []
    
    # 模拟不同策略下的期望总成本
    for i in range(num_simulations):
        # 随机次品率
        part1_defects = part1_defect_rate_dist[i]
        part2_defects = part2_defect_rate_dist[i]
        product_defects = product_defect_rate_dist[i]
        
        # 计算不同策略下的成本
        detection_part1 = detection_cost_part1 if np.random.rand() > part1_defects else 0
        detection_part2 = detection_cost_part2 if np.random.rand() > part2_defects else 0
        detection_product = detection_cost_product if np.random.rand() > product_defects else 0
        
        # 合格品的收益和次品的损失
        revenue = (1 - product_defects) * revenue_per_product
        loss = product_defects * (replacement_loss + reputation_loss)
        
        # 总成本计算
        total_cost = detection_part1 + detection_part2 + detection_product + assembly_cost - revenue + loss
        total_costs.append(total_cost)
    
    # 返回平均成本
    return np.mean(total_costs)

# 调用蒙特卡洛模拟求解
expected_cost = monte_carlo_simulation()
print(f"蒙特卡洛模拟下的期望成本: {expected_cost}")


蒙特卡洛模拟下的期望成本: -34.63401378355377
